In [2]:
!pip install gdown

In [3]:
!pip install keras-tuner

In [4]:
#imports
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import itertools
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import time
from tensorflow.keras.callbacks import Callback
from keras.callbacks import EarlyStopping
import gdown
from keras.regularizers import l2
import keras_tuner as kt
from tensorflow.keras.models import save_model

#File IDs for the CSV files
file_id_train = '1Jtx8g3kartST38gtRKcqQf9RxGNE2l76'  
file_id_test = '1Yz-nIiuk7trjFBZQC7vu19gguzIPX255'   

#URLs for downloading the CSV files
csv_file_url_train = f'https://drive.google.com/uc?id={file_id_train}'
csv_file_url_test = f'https://drive.google.com/uc?id={file_id_test}'

#Download CSV files
gdown.download(csv_file_url_train, 'sign_mnist_train.csv', quiet=False)
gdown.download(csv_file_url_test, 'sign_mnist_test.csv', quiet=False)

#Read CSV files
train_data = pd.read_csv('sign_mnist_train.csv')
test_data = pd.read_csv('sign_mnist_test.csv')

#Concatenate train and test data
combined_data = pd.concat([train_data, test_data], ignore_index=True)

print(combined_data.shape)
combined_data.head()

Downloading...
From: https://drive.google.com/uc?id=1Jtx8g3kartST38gtRKcqQf9RxGNE2l76
To: /Users/sofiautoft/ML/ASLproject/sign_mnist_train.csv
100%|██████████| 83.3M/83.3M [00:05<00:00, 14.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Yz-nIiuk7trjFBZQC7vu19gguzIPX255
To: /Users/sofiautoft/ML/ASLproject/sign_mnist_test.csv
100%|██████████| 21.8M/21.8M [00:01<00:00, 17.2MB/s]


(34627, 785)


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [5]:
#split data into features (X) and labels (y)
X = combined_data.drop(columns=['label'])
y = combined_data['label']

#first, split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state = 123)

#then, split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

#create dictionary to go from number -> alphabetical letter
num_to_letters = {}
for num in range(0, 25):
    letter = chr(num + 65)
    num_to_letters[num] = letter

Y_train_labels = [num_to_letters[num] for num in y_train]

Y_train_labels = sorted(Y_train_labels)

In [6]:
#normalize the data (must be between 0 and 1 to feed our model)
X_train = X_train / 255.0
X_test = X_test / 255.0
X_val = X_val / 255.0

#reshape the data into 28x28
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)
X_val = X_val.values.reshape(-1,28,28,1)

#one hot encoding
Y_train_encoded = to_categorical(y_train, num_classes = 25)
Y_test_encoded = to_categorical(y_test, num_classes=25)
Y_val_encoded =  to_categorical(y_val, num_classes=25)

In [7]:
#Define the unique identifiers for kernel sizes
kernel_sizes = [(3, 3), (5, 5), (8, 8)]
kernel_size_names = ['small', 'medium', 'large']

#Define the model-building function
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(filters=hp.Choice('filters', values=[8, 16, 32]),
                     kernel_size=kernel_sizes[kernel_size_names.index(hp.Choice('kernel_size', values=kernel_size_names))],
                     padding='same',
                     activation='relu',
                     input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout_1', values=[0.25, 0.3, 0.35])))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16,
                     kernel_size=kernel_sizes[kernel_size_names.index(hp.Choice('kernel_size_2', values=kernel_size_names))],
                     padding='same',
                     activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout_2', values=[0.25, 0.3, 0.35])))
    model.add(Flatten())
    model.add(Dense(256, activation="relu", kernel_regularizer=l2(0.001)))
    model.add(Dropout(hp.Choice('dropout_3', values=[0.3, 0.5, 0.6])))
    model.add(Dense(25, activation="softmax"))
    
    #Define optimizer
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.01]))
    #Compile the model
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model

#Instantiate the Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='my_project'
)

#Perform Hyperparameter Search
tuner.search(X_train, Y_train_encoded, epochs=50, validation_split = 0.3, callbacks=[EarlyStopping()])

#Get the Best Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

#Compile the Model with Best Hyperparameters
model = tuner.hypermodel.build(best_hps)
model.compile(optimizer=model.optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#Augment Data
datagen = ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False, zca_whitening=False, 
        rotation_range = 10,  #randomly rotate images 5 degrees
        zoom_range = 0.3, #randomly zoom in 10%
        width_shift_range= 0.3,  #randomly shift images horizontally 10%
        height_shift_range= 0.3,  #randomly shift images vertically 10%
        horizontal_flip=False,  #randomly flip images set to false - could this affect the sign if True (?)
        vertical_flip=False)  #randomly flip images set to false - could this affect the sign if True (?)

datagen.fit(X_train)

#Keep track of Training Time
class TrainingTimeCallback(Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()

    def on_train_end(self, logs=None):
        end_time = time.time()
        training_duration = end_time - self.start_time
        print("Training duration:", training_duration, "seconds")

#create an instance of the custom callback
time_callback = TrainingTimeCallback()

#Train the Model

data = X_train.shape[0]
epochs = 118 #whole number options = 1, 2, 59, 118, 143, 286, 8483, 16966
batch_size = int(data / epochs)

#calculate the steps per epoch
steps_per_epoch = int(X_train.shape[0] / batch_size)

#fit the model
history = model.fit(datagen.flow(X_train, Y_train_encoded, batch_size=batch_size), epochs=epochs, 
                    validation_data=(X_test, Y_test_encoded), steps_per_epoch=steps_per_epoch, callbacks=[time_callback], shuffle = True)


Reloading Tuner from my_dir/my_project/tuner0.json
Epoch 1/118
118/118 [==============================] - 10s 83ms/step - loss: 3.6473 - accuracy: 0.0771 - val_loss: 3.5504 - val_accuracy: 0.0622
Epoch 2/118
118/118 [==============================] - 10s 84ms/step - loss: 3.2719 - accuracy: 0.1181 - val_loss: 3.2854 - val_accuracy: 0.1347
Epoch 3/118
118/118 [==============================] - 10s 83ms/step - loss: 3.0210 - accuracy: 0.1694 - val_loss: 2.7978 - val_accuracy: 0.2492
Epoch 4/118
118/118 [==============================] - 9s 79ms/step - loss: 2.7426 - accuracy: 0.2276 - val_loss: 2.7070 - val_accuracy: 0.2357
Epoch 5/118
118/118 [==============================] - 10s 82ms/step - loss: 2.4816 - accuracy: 0.2868 - val_loss: 1.9192 - val_accuracy: 0.4717
Epoch 6/118
118/118 [==============================] - 9s 79ms/step - loss: 2.2508 - accuracy: 0.3483 - val_loss: 1.6697 - val_accuracy: 0.5333
Epoch 7/118
118/118 [==============================] - 10s 85ms/step - loss: 2.08

In [8]:
#Validation Accuracy and Loss
yPred = model.predict(X_val)
cce = tf.keras.losses.CategoricalCrossentropy()
loss = cce(Y_val_encoded, yPred).numpy()

#Validation Loss
metric = tf.keras.metrics.CategoricalAccuracy()
metric.update_state(Y_val_encoded, yPred)
accuracy = metric.result().numpy()

print("The accuracy of our model on the validation data is", accuracy, "\nThe loss of our model on the validation data is", loss)

228/228 [==============================] - 1s 4ms/step
The accuracy of our model on the validation data is 0.9682343 
The loss of our model on the validation data is 0.12756321


In [9]:
#save trained model
model.save("CNN_model.h5")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
